# Notebook 05: numpy and matplotlib

## Content
- numpy.ndarray creation and usage
- basic plotting with matplotlib

## Remember jupyter notebooks
- To run the currently highlighted cell, hold <kbd>&#x21E7; Shift</kbd> and press <kbd>&#x23ce; Enter</kbd>.
- To get help for a specific function, place the cursor within the function's brackets, hold <kbd>&#x21E7; Shift</kbd>, and press <kbd>&#x21E5; Tab</kbd>.

## A notebook "preamble"
The forst code block prepares our notebook by specifying how to render plots and importing two required packages.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

## ndarray: numpy's central data structure

In [ ]:
a = list(range(5))
print(a, type(a))

b = np.asarray(a)
print(b, type(b))

In [ ]:
a = [[0, 1, 2], [3, 4, 5]]
print(a, type(a))

b = np.asarray(a)
print(b, type(b))

In [ ]:
print(b.size)
print(b.ndim)
print(b.shape)
print(b.dtype)

How does `numpy` select the appropriate `dtype`?

In [ ]:
np.asarray([0, 1])

In [ ]:
np.asarray([0, 1, 2.0])

In [ ]:
np.asarray([0, 1, 2.0, 3+0j])

In [ ]:
np.asarray([0, 1, 2.0, 3+0j, 'four'])

In [ ]:
np.asarray([0, 1, 2.0, 3+0j, 'four', None])

Creating arrays with "default" values.

In [ ]:
a = np.zeros((2, 3, 4), dtype=np.float64)
print(a)
print(a.size, a.ndim, a.shape)

In [ ]:
print(np.ones((4, 3, 2), dtype=np.int))

In [ ]:
a = np.arange(16)
print(a)

The `shape` can be changed...

In [ ]:
a = a.reshape(-1, 4)
print(a)

In [ ]:
a.reshape(-1)

... and the `dtype`, too:

In [ ]:
a = a.astype(np.float64)
print(a)

You can index like a nested `list`/`tuple`...

In [ ]:
print(a[-1][0])

... or via the `numpy` way:

In [ ]:
print(a[-1, 0])

Slicing works, too:

In [ ]:
print(a[:, 0])
print(a[0, :])

Even for assignments!

In [ ]:
a[:, -1] *= -1
print(a)

You can (implicitly) iterate over the first index:

In [ ]:
for b in a:
    print(b)

In [ ]:
for b in a.T:
    print(b)

Can we add/subtract/... somthing to/from an array?

In [ ]:
a = np.arange(5)
print(a)

In [ ]:
print(a + 1)

In [ ]:
print(a - 1.0)

In [ ]:
print(a * 1+0j)

In [ ]:
print(a / 2)

In [ ]:
print(a // 2)

In [ ]:
print(a**2)

In [ ]:
print(a % 2)

What about adding/... two arrays?

In [ ]:
b = np.ones(a.size) * 2
print(b)

In [ ]:
print(a + b)

In [ ]:
print(a - b)

In [ ]:
print(a * b)

In [ ]:
print(a / b)

We can evaluate function on the whole array in one step:

In [ ]:
print(np.sqrt(a))

In [ ]:
print(np.exp(a))

In [ ]:
print(np.log(a + 1))

In [ ]:
print(np.sin(a))

Summations/multiplications over the whole array or selected axes are possible:

In [ ]:
a = np.ones((3, 5))
print(a)
print(a.sum())
print(a.sum(axis=0))
print(a.sum(axis=1))

In [ ]:
a = np.ones((3, 5)) * 2
print(a.prod())
print(a.prod(axis=0))
print(a.prod(axis=1))

In [ ]:
a = np.ones((5, 3))
print(np.sqrt(np.sum(a**2, axis=-1)))

In [ ]:
print(np.linalg.norm(a, axis=-1))

## Mini project refactoring

**Exercise**: refactor the `scalar_product(a, b)` function to make use of `numpy`.

In [ ]:
def scalar_product(a, b):
    return np.sum(np.asarray(a) * np.asarray(b))

In [ ]:
assert scalar_product([0] * 100, [1] * 100) == 0
assert scalar_product([1] * 100, [1, -1] * 50) == 0
assert scalar_product([1] * 100, range(100)) == 99 * 50

**Exercise**: refactor the `mean(a)` function.

In [ ]:
def mean(a):
    if len(a) == 0:
        return 0
    return np.sum(a) / len(a)

In [ ]:
assert mean(range(100)) == 99 * 0.5
assert mean([]) == 0
assert mean([1] * 1000) == 1

**Exercise**: refactor the `linear_regression(x, y)` function.

In [ ]:
def linear_regression(x_values, y_values):
    x_mean, y_mean = mean(x_values), mean(y_values)
    x = np.asarray(x_values) - x_mean
    y = np.asarray(y_values) - y_mean
    slope = scalar_product(x, y) / np.sum(x**2)
    const = y_mean - slope * x_mean
    return slope, const

In [ ]:
x = [10, 14, 16, 15, 16, 20]
y = [ 1,  3,  5,  6,  5, 11]
slope, const = linear_regression(x, y)
assert 0.97 < slope < 0.99
assert -9.72 < const < -9.70

## Vectorisation
Computing distances can be an expensive task as it is $\mathcal{O}(N^2)$.

In [ ]:
def get_distances(coordinates):
    distances = np.zeros((len(coordinates), len(coordinates)))
    for i in range(len(coordinates)):
        for j in range(len(coordinates)):
            distances[i, j] = np.linalg.norm(
                coordinates[i] - coordinates[j],
                axis=-1)
    return distances


coordinates = np.random.rand(1000, 3)
%timeit get_distances(coordinates)

We can, of course, exploit symmetry:

In [ ]:
def get_distances2(coordinates):
    distances = np.zeros((len(coordinates), len(coordinates)))
    for i in range(1, len(coordinates)):
        for j in range(i):
            distances[i, j] = np.linalg.norm(
                coordinates[i] - coordinates[j],
                axis=-1)
            distances[j, i] = distances[i, j]
    return distances


%timeit get_distances2(coordinates)

But **vectorisation** is much faster and easier to write:

In [ ]:
def get_distances3(coordinates):
    return np.linalg.norm(
        coordinates[:, None, :] - coordinates[None, :, :],
        axis=-1)


%timeit get_distances3(coordinates)

In the above example, we traded loops against higher memory requirement. To see how that works, let's look at what a `None` does for array indexing:

In [ ]:
a = np.arange(5)
print(a)

In [ ]:
print(a[:, None])

In [ ]:
print(a[None, :])

In [ ]:
print(a[None, :, None])

In [ ]:
a = np.arange(16).reshape(4, -1)
b = a[1:-1, 1:-1]
print(a)
print(b)

In [ ]:
b *= -1
print(a)

## Plotting
Let's try to visualise a function:

In [ ]:
x = np.linspace(-np.pi, np.pi, 100)
s = np.sin(x)

plt.plot(x, s)
plt.xlabel('$x$ / rad', fontsize=15)
plt.ylabel('$\sin(x)$', fontsize=15)

In [ ]:
c = np.cos(x)

plt.plot(x, s, label='sin')
plt.plot(x, c, label='cos')
plt.xlabel('$x$ / rad', fontsize=15)
plt.legend(fontsize=15)

Let's use that for the linear regression problem:

In [ ]:
x = np.asarray([10, 14, 16, 15, 16, 20])
y = np.asarray([ 1,  3,  5,  6,  5, 11])
slope, const = linear_regression(x, y)

plt.scatter(x, y)
plt.plot(x, const + slope * x)
plt.xlabel('$x$')
plt.ylabel('$y$')

And now for a bigger version of the same problem:

In [ ]:
x = np.sort(np.random.rand(1000))
true_slope, true_const = np.random.uniform(low=0.2, high=1.0, size=2)
y = true_const + true_slope * x + np.random.randn(x.size) * 0.1

slope, const = linear_regression(x, y)

fig, ax = plt.subplots(figsize=(6, 4))
ax.scatter(x, y, marker='p', s=0.5, label='observation')
ax.plot(x, const + slope * x, linewidth=3, label='regression')
ax.plot(x, true_const + true_slope * x, '--', linewidth=3, label='true')
ax.legend(fontsize=15)
ax.set_xlabel('$x$', fontsize=15)
ax.set_ylabel('$y$', fontsize=15)
fig.tight_layout()

And finally an example where the linear model will (most likely) fail:

In [ ]:
x = np.sort(10 * np.random.rand(1000) - 5)
a, b, c = np.random.uniform(low=0.2, high=1.0, size=3)
y = a + b * x + c * x**2 + np.random.randn(x.size)

slope, const = linear_regression(x, y)

fig, ax = plt.subplots(figsize=(6, 4))
ax.scatter(x, y, marker='p', s=0.5, label='observation')
ax.plot(x, const + slope * x, linewidth=3, label='regression')
ax.legend(fontsize=15)
ax.set_xlabel('$x$', fontsize=15)
ax.set_ylabel('$y$', fontsize=15)
fig.tight_layout()